### Libraries

In [3]:
# Load Libraries
import os
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from datetime import date, timedelta,datetime
import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:0,.1f}'.format 
sns.set_style("white")

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm



print( mpl.__version__)

3.5.2


### Workspace

In [5]:
%cd /home/analysis_lake

/home/analysis_lake


### Day Check

In [6]:
from pytz import timezone
from datetime import datetime
today = datetime.now(timezone('Asia/Seoul'))
print(today)

2022-07-11 17:47:06.830803+09:00


In [7]:
# today = date.today()
yesterday = today - timedelta(1)
this_week = today.isocalendar()[1]

print(today.strftime('%Y-%m-%d'))
print(yesterday.strftime('%Y-%m-%d'))
print(this_week)

2022-07-11
2022-07-10
28


### Big Query

In [8]:
project_id = 'kr-co-vcnc-tada'

In [9]:
import pandas as pd
import pydata_google_auth
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/bigquery'
]

credentials = pydata_google_auth.get_user_credentials(
  SCOPES, auth_local_webserver=True)

In [10]:

query = """ 

  SELECT 
    FORMAT_DATE("%G-%V",date_kr) as year_week, 
    count(ride_id) as ride_cnt,
    count(distinct(vehicle_id)) as vehicle_cnt,
    count(distinct(rider_id)) as rider_cnt
  FROM tada_store.ride_base A
  WHERE 
    dropped_off_at_kr is not Null AND 
    type in ('NXT')
  GROUP BY 1
  ORDER BY 1
  
 """
                
df = pd.read_gbq(query=query, project_id=project_id, credentials=credentials, dialect='standard')


In [11]:
df.head()

,year_week,ride_cnt,vehicle_cnt,rider_cnt
0,2021-47,2442,104,2150
1,2021-48,11673,213,8264
2,2021-49,15663,235,10072
3,2021-50,16239,242,10739
4,2021-51,14985,244,9642


In [12]:
df_analysis = df.copy()

In [13]:
df_analysis['ride_per_rider'] = df_analysis['ride_cnt']/df_analysis['rider_cnt']
df_analysis['ride_per_vehicle'] = df_analysis['ride_cnt']/df_analysis['ride_cnt'] 
df_analysis['ride_per_verhicle'] = df_analysis['ride_cnt']/df_analysis['rider_cnt'] 

In [14]:
df_analysis.head()

,year_week,ride_cnt,vehicle_cnt,rider_cnt,ride_per_rider,ride_per_vehicle,ride_per_verhicle
0,2021-47,2442,104,2150,1.1,1.0,1.1
1,2021-48,11673,213,8264,1.4,1.0,1.4
2,2021-49,15663,235,10072,1.6,1.0,1.6
3,2021-50,16239,242,10739,1.5,1.0,1.5
4,2021-51,14985,244,9642,1.6,1.0,1.6


In [52]:
df_analysis.head()

,ride_cnt,vehicle_cnt,rider_cnt,ride_per_rider,ride_per_vehicle,rider_per_vehicle
year_week,,,,,,
2021-47,2442,104,2150,1.1,23.5,20.7
2021-48,11673,213,8264,1.4,54.8,38.8
2021-49,15663,235,10072,1.6,66.7,42.9
2021-50,16239,242,10739,1.5,67.1,44.4
2021-51,14985,244,9642,1.6,61.4,39.5
